In [1]:
import tensorflow as tf
import os
print(tf.__version__)

2.5.0


In [2]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score

from seqeval.metrics import classification_report,accuracy_score,f1_score
from tqdm import tqdm,trange

In [3]:
df_data = pd.read_csv("ner_dataset2.csv",sep=",",encoding="latin1").fillna(method='ffill')
df_data.shape

(5683, 3)

In [4]:
tag_list=df_data.Tag.unique()
tag_list

array(['Name', 'Position', 'Company', 'Other', 'Address'], dtype=object)

In [5]:

from sklearn.model_selection import train_test_split
x_train,x_test=train_test_split(df_data,test_size=0.20,shuffle=False)

In [6]:
x_train.shape,x_test.shape

((4546, 3), (1137, 3))

In [7]:
agg_func = lambda s: [ [w,t] for w,t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]

In [8]:
x_train_grouped = x_train.groupby("ID").apply(agg_func)
x_test_grouped = x_test.groupby("ID").apply(agg_func)

In [9]:
x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values]
x_test_sentences = [[s[0] for s in sent] for sent in x_test_grouped.values]


In [10]:
x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values]
x_test_tags = [[t[1] for t in tag] for tag in x_test_grouped.values]

In [11]:
np.shape(x_train_tags),np.shape(x_test_tags)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


((331,), (86,))

In [12]:
MAX_LENGTH=200
BERT_MODEL='bert-base-uncased'

BATCH_SIZE=32

pad_token=0,
pad_token_segment_id=0,
sequence_a_segment_id=0,

In [13]:
# pip install transformers

In [14]:
from transformers import (
    TF2_WEIGHTS_NAME,
    BertConfig,
    BertTokenizer,
    TFBertForTokenClassification,
    create_optimizer)

In [15]:
MODEL_CLASSES = {"bert": (BertConfig, TFBertForTokenClassification, BertTokenizer)}

In [16]:
label_map = {label: i+1 for i, label in enumerate(tag_list)}

In [17]:
num_labels = len(tag_list) + 1
num_labels

6

In [18]:
pad_token_label_id = 0

In [19]:
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

In [20]:
config = config_class.from_pretrained(BERT_MODEL,num_labels=num_labels)

In [21]:
tokenizer = tokenizer_class.from_pretrained(BERT_MODEL,do_lower_case=False)

In [22]:
model = model_class.from_pretrained(
                BERT_MODEL,
                from_pt=bool(".bin" in BERT_MODEL),
                config=config)

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
 model.layers[-1].activation = tf.keras.activations.softmax

In [24]:
# from keras.preprocessing.sequence import pad_sequences
# max_seq_length =128

# def convert_to_input(sentences,tags):
#   input_id_list,attention_mask_list,token_type_id_list=[],[],[]
#   label_id_list=[]
  
#   for x,y in tqdm(zip(sentences,tags),total=len(tags)):
  
#     tokens = []
#     label_ids = []

#     for word, label in zip(x, y):
#       word_tokens = tokenizer.tokenize(word)
#       tokens.extend(word_tokens)
#       # Use the real label id for the first token of the word, and padding ids for the remaining tokens
#       label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

  
#     special_tokens_count =  2
#     if len(tokens) > max_seq_length - special_tokens_count:
#       tokens = tokens[: (max_seq_length - special_tokens_count)]
#       label_ids = label_ids[: (max_seq_length - special_tokens_count)]

#     label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
#     inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

#     input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
#     attention_masks = [1] * len(input_ids)

#     attention_mask_list.append(attention_masks)
#     input_id_list.append(input_ids)
#     token_type_id_list.append(token_type_ids)

#     label_id_list.append(label_ids)

#   return input_id_list,token_type_id_list,attention_mask_list,label_id_list


In [25]:
from keras.preprocessing.sequence import pad_sequences
max_seq_length =200

def convert_to_input(sentences,tags):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]
  
  for x,y in tqdm(zip(sentences,tags),total=len(tags)):
  
    tokens = []
    label_ids = []

    for word, label in zip(x, y):
      word_tokens = tokenizer.tokenize(word)
      tokens.extend(word_tokens)
      # Use the real label id for the first token of the word, and padding ids for the remaining tokens
      label_ids.extend([label_map[label]]  * (len(word_tokens)))

  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]
      label_ids = label_ids[: (max_seq_length - special_tokens_count)]

    label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

    label_id_list.append(label_ids)

  return input_id_list,token_type_id_list,attention_mask_list,label_id_list
# 

In [26]:
input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)

100%|███████████████████████████████████████████████████████████████████████████████| 331/331 [00:00<00:00, 534.96it/s]


In [27]:
input_ids_test,token_ids_test,attention_masks_test,label_ids_test=convert_to_input(x_test_sentences,x_test_tags)

100%|█████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 597.34it/s]


In [28]:
input_ids_train = pad_sequences(input_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_train = pad_sequences(token_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_train = pad_sequences(attention_masks_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_train = pad_sequences(label_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

In [29]:
np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train),

((331, 200), (331, 200), (331, 200), (331, 200))

In [30]:
input_ids_test = pad_sequences(input_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_test = pad_sequences(token_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_test = pad_sequences(attention_masks_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_test = pad_sequences(label_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

In [31]:
np.shape(input_ids_test),np.shape(token_ids_test),np.shape(attention_masks_test),np.shape(label_ids_test),

((86, 200), (86, 200), (86, 200), (86, 200))

In [32]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

train_ds = tf.data.Dataset.from_tensor_slices((input_ids_train,attention_masks_train,token_ids_train,label_ids_train)).map(example_to_features).shuffle(1000).batch(32).repeat(5)


test_ds=tf.data.Dataset.from_tensor_slices((input_ids_test,attention_masks_test,token_ids_test,label_ids_test)).map(example_to_features).batch(1)


In [33]:
for x,y in test_ds.take(10):
  print(x,y)

{'input_ids': <tf.Tensor: shape=(1, 200), dtype=int32, numpy=
array([[  101,   100,   100,  1019,  1011,  1015,  1010,   100,  1011,
          100,  1016,  1011, 16480,  4168,  1010,   100,  1011, 13970,
         1010,   100, 10550,  1011,  3770,  2581,  2581,  1010,   100,
          100,  2000,  1996,  3795,  2204,  1012,   100,  1009,  6282,
         1011,  1017,  1011,  4090,  2683,  2581,  1011,  6275, 17465,
          100,  1009,  6282,  1011,  1017,  1011,  4090,  2683,  2581,
         1011, 25374,  2487,   100,  1011,  5653,  1024, 17823, 22591,
         3406,  1011,  7632,  7352,  1030, 23333, 20760,  1012,  2522,
         1012, 16545,   100,  1011,  5653,  1024, 17823, 22591,  3406,
         1011,  7632,  7352,  1030, 23333, 20760,  1012,  2522,  1012,
        16545,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [34]:
model.summary()

Model: "tf_bert_for_token_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108891648 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  4614      
Total params: 108,896,262
Trainable params: 108,896,262
Non-trainable params: 0
_________________________________________________________________


In [35]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


In [36]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [41]:
# history = model.fit(train_ds, epochs=1, validation_data=test_ds)

In [40]:
model.load_weights('Card_model_weights.h5')

In [42]:
model.evaluate(test_ds)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\backend.py:4929: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


86/86 [==============================] - 32s 330ms/step - loss: 1.2570 - accuracy: 0.4688


[1.2570463418960571, 0.4688372015953064]

In [ ]:
model.save_weights('Card_model_weights.h5')

In [ ]:
pred=model.predict(test_ds)

In [ ]:
type(pred)


In [ ]:
pred.logits[1]

In [39]:
tokenizer.vocab_size

30522